In [1]:
import joblib
import pathlib2 as pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from collections import Counter

from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD

In [2]:
picspath=pathlib.Path.cwd().joinpath('pics')

In [3]:
# load pipelines
#titles
nmf_titles_model=joblib.load('models/topicmodeling_nmf_titles/topicmodeling_nmf_titles.pkl')
lsa_titles_model=joblib.load('models/topicmodeling_lsa_titles/topicmodeling_lsa_titles.pkl')
lda_titles_model=joblib.load('models/topicmodeling_lda_titles/topicmodeling_lda_titles.pkl')
#abstracts
nmf_abstracts_model=joblib.load('models/topicmodeling_nmf_abstracts/topicmodeling_nmf_abstracts.pkl')
lsa_abstracts_model=joblib.load('models/topicmodeling_lsa_abstracts/topicmodeling_lsa_abstracts.pkl')
lda_abstracts_model=joblib.load('models/topicmodeling_lda_abstracts/topicmodeling_lda_abstracts.pkl')

In [4]:
#load data
datapath=pathlib.Path.cwd().joinpath('cleaned_data')
data=pd.read_csv(datapath.joinpath("cleaned_data.csv").as_posix(),sep=';')
data.head()

C:\Users\lekle\AppData\Local\Temp\ipykernel_33552\104726719.py:3: DtypeWarning: Columns (23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(datapath.joinpath("cleaned_data.csv").as_posix(),sep=';')


,Unnamed: 0,Lens ID,Title,Date Published,Publication Year,Publication Type,Source Title,ISSNs,Publisher,Source Country,...,Microsoft Academic ID,PMCID,Citing Patents Count,References,Citing Works Count,Is Open Access,Open Access License,Open Access Colour,title_languages,abstract_languages
0,0,000-008-268-360-004,Day-Ahead Solar Irradiance Forecasting in a Tr...,2015-07-27,2015,journal article,Journal of Solar Energy Engineering,01996231; 15288986,ASME International,United States,...,2131632891,NaN,0,001-815-086-488-477; 007-447-979-011-061; 025-...,33,False,NaN,NaN,en,en
1,2,000-042-629-788-452,Assessment of a falling solid particle receive...,NaN,2015,journal article,Solar Energy,0038092x,Elsevier BV,United Kingdom,...,2067658121,NaN,0,002-994-869-371-657; 005-360-904-860-363; 008-...,62,True,NaN,green,en,en
2,4,000-062-946-383-801,Boron-Doped Silicon Diatom Frustules as a Phot...,2015-07-30,2015,journal article,ACS applied materials & interfaces,19448252; 19448244,American Chemical Society (ACS),United States,...,2410807451,NaN,0,004-049-858-705-806; 007-090-004-757-278; 011-...,26,False,NaN,NaN,en,en
3,5,000-063-768-273-718,Renewable energy management through microgrid ...,NaN,2015,journal article,Energy Reports,23524847,Elsevier BV,United Kingdom,...,1193286937,NaN,0,004-077-674-704-399; 005-081-834-054-355; 011-...,79,True,"CC BY, CC BY-NC-ND",gold,en,en
4,11,000-138-869-752-834,Thermal and electrical analysis of a linear pa...,2015-12-01,2015,NaN,NaN,NaN,NaN,NaN,...,3210564331,NaN,0,NaN,0,False,NaN,NaN,en,en


In [7]:
data.columns

Index(['Unnamed: 0', 'Lens ID', 'Title', 'Date Published', 'Publication Year',
       'Publication Type', 'Source Title', 'ISSNs', 'Publisher',
       'Source Country', 'Author/s', 'Abstract', 'Volume', 'Issue Number',
       'Start Page', 'End Page', 'Fields of Study', 'Keywords', 'MeSH Terms',
       'Chemicals', 'Funding', 'Source URLs', 'External URL', 'PMID', 'DOI',
       'Microsoft Academic ID', 'PMCID', 'Citing Patents Count', 'References',
       'Citing Works Count', 'Is Open Access', 'Open Access License',
       'Open Access Colour', 'title_languages', 'abstract_languages'],
      dtype='object')

In [8]:
nmf_title=nmf_titles_model.transform(data.Title)

In [9]:
nmf_title[0]

array([1.18446645e-04, 0.00000000e+00, 0.00000000e+00, 1.66286036e-03,
       0.00000000e+00, 1.12687752e-03, 0.00000000e+00, 1.41483676e-03,
       1.80118299e-05, 0.00000000e+00, 6.79809383e-04, 8.33392930e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.85547393e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.95845416e-02, 0.00000000e+00,
       1.74945353e-03])

In [5]:
def get_topic_destribution(model,datacolumn):
    if any(isinstance(step[1], NMF) for step in model.steps):
        W_text_matrix=model.transform(datacolumn)
        x=W_text_matrix.sum(axis=0)/W_text_matrix.sum()*100.0
    elif any(isinstance(step[1], TruncatedSVD) for step in model.steps):
        x=model.named_steps['lsa'].components_.sum(axis=1) / model.named_steps['lsa'].components_.sum() * 100.0
    elif any(isinstance(step[1], LatentDirichletAllocation) for step in model.steps):
        W_text_matrix=model.transform(datacolumn)
        x=W_text_matrix.sum(axis=0) / W_text_matrix.sum(axis=0).sum() * 100.0
    return x

def vis_w_matrix(w_maxtrix,modeltitle,file):
    x=[f"Topic {x}" for x in range(len(w_maxtrix))]
    y=w_maxtrix
    plt.figure(figsize=(10,6))
    plt.bar(x, y, align='center', width=0.5, color='seagreen')
    plt.xlabel('Topics')
    # plt.xticks(rotation=90, ha='right')
    plt.xticks(rotation='vertical')
    plt.ylabel('Topic Distribution in %')
    plt.title(f'{modeltitle}')
    plt.legend()
    plt.savefig(file, bbox_inches='tight')
    plt.close()
    # plt.show()

In [11]:
w_maxtrix=get_topic_destribution(nmf_titles_model,data.Title)
w_maxtrix

array([ 2.46687693,  6.52306163,  2.20900128,  4.22039214,  3.22088125,
        4.05974457,  2.55362659,  2.29899261,  3.0627456 ,  2.5978215 ,
        4.62269574,  4.91056724,  4.08269504,  3.36235785,  2.20072089,
        3.70214951,  3.27149542,  3.22983494,  3.14825647,  3.30880385,
        5.0926115 ,  3.26887673, 10.95987893,  7.58010382,  4.04580797])

In [12]:
w_maxtrix.sum()

100.00000000000001

In [13]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,data.Title),"NMF Title",picspath.joinpath('nmf_title.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,data.Title),"LSA Title",picspath.joinpath('lsa_title.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,data.Title),"LDA Title",picspath.joinpath('lda_title.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [14]:
vis_w_matrix(get_topic_destribution(nmf_abstracts_model,data.Abstract),"NMF Abstract",picspath.joinpath('nmf_abstract.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,data.Abstract),"LSA Abstract",picspath.joinpath('lsa_abstract.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,data.Abstract),"LDA Abstract",picspath.joinpath('lda_abstract.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


# vis w/ filtered data: year

In [15]:
data2015=data[data['Publication Year']==2015]
data2024=data[data['Publication Year']==2024]
# data2015.head()

In [16]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,data2015.Title),"NMF Title 2015",picspath.joinpath('nmf_title2015.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,data2015.Title),"LSA Title 2015",picspath.joinpath('lsa_title2015.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,data2015.Title),"LDA Title 2015",picspath.joinpath('lda_title2015.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,data2015.Abstract),"NMF Abstract 2015",picspath.joinpath('nmf_abstract2015.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,data2015.Abstract),"LSA Abstract 2015",picspath.joinpath('lsa_abstract2015.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,data2015.Abstract),"LDA Abstract 2015",picspath.joinpath('lda_abstract2015.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [17]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,data2024.Title),"NMF Title 2024",picspath.joinpath('nmf_title2024.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,data2024.Title),"LSA Title 2024",picspath.joinpath('lsa_title2024.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,data2024.Title),"LDA Title 2024",picspath.joinpath('lda_title2024.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,data2024.Abstract),"NMF Abstract 2024",picspath.joinpath('nmf_abstract2024.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,data2024.Abstract),"LSA Abstract 2024",picspath.joinpath('lsa_abstract2024.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,data2024.Abstract),"LDA Abstract 2024",picspath.joinpath('lda_abstract2024.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


# vis w/ filtered data: Journal

In [18]:
dataSpringer=data[data['Publisher']=="Springer Science and Business Media LLC"]
dataElsevier=data[data['Publisher']=="Elsevier BV"]

In [19]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,dataSpringer.Title),"NMF Title Springer Science and Business Media LLC",picspath.joinpath('nmf_titleSpringer.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,dataSpringer.Title),"LSA Title Springer Science and Business Media LLC",picspath.joinpath('lsa_titleSpringer.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,dataSpringer.Title),"LDA Title Springer Science and Business Media LLC",picspath.joinpath('lda_titleSpringer.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,dataSpringer.Abstract),"NMF Abstract Springer Science and Business Media LLC",picspath.joinpath('nmf_abstractSpringer.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,dataSpringer.Abstract),"LSA Abstract Springer Science and Business Media LLC",picspath.joinpath('lsa_abstractSpringer.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,dataSpringer.Abstract),"LDA Abstract Springer Science and Business Media LLC",picspath.joinpath('lda_abstractSpringer.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [20]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,dataElsevier.Title),"NMF Title Elsevier BV",picspath.joinpath('nmf_titleElsevier.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,dataElsevier.Title),"LSA Title Elsevier BV",picspath.joinpath('lsa_titleElsevier.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,dataElsevier.Title),"LDA Title Elsevier BV",picspath.joinpath('lda_titleElsevier.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,dataElsevier.Abstract),"NMF Abstract Elsevier BV",picspath.joinpath('nmf_abstractElsevier.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,dataElsevier.Abstract),"LSA Abstract Elsevier BV",picspath.joinpath('lsa_abstractElsevier.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,dataElsevier.Abstract),"LDA Abstract Elsevier BV",picspath.joinpath('lda_abstractElsevier.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


# vis w/ filtered data: Authors

In [6]:
data = data[data['Author/s'].apply(lambda x: isinstance(x, str))]
data['authors_cleaned']=data['Author/s'].apply(lambda x: x.split(';'))

In [22]:
data.authors_cleaned.head()

0    [Aloysius W. Aryaputera,  Dazhi Yang,  Wilfred...
1    [Birgit Gobereit,  Lars Amsbeck,  Reiner Buck,...
2    [Soundarrajan Chandrasekaran,  Thomas J. Macdo...
3                    [null Zaheeruddin,  Munish Manas]
4                         [Farid Safari,  Abtin Ataei]
Name: authors_cleaned, dtype: object

In [7]:
all_authors= [item for sublist in data.authors_cleaned for item in sublist]
author_counts = Counter(all_authors)
#  Bhim Singh': 639; Liang Li': 464
# hat noch Leerzeichen vorneab, kann ich mit leben
# author_counts

In [24]:
# data['authors_cleaned'].value_counts()

In [8]:
dataSingh=data[data.authors_cleaned.apply(lambda x: ' Bhim Singh' in x)]
dataLi=data[data.authors_cleaned.apply(lambda x: ' Liang Li' in x)]
dataMeyer=data[data.authors_cleaned.apply(lambda x: ' J-P. Meyer' in x)]

In [26]:
# dataSingh.head()

In [27]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,dataSingh.Title),"NMF Title Bhim Singh",picspath.joinpath('nmf_titleSingh.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,dataSingh.Title),"LSA Title Bhim Singh",picspath.joinpath('lsa_titleSingh.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,dataSingh.Title),"LDA Title Bhim Singh",picspath.joinpath('lda_titleSingh.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,dataSingh.Abstract),"NMF Abstract Bhim Singh",picspath.joinpath('nmf_abstractSingh.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,dataSingh.Abstract),"LSA Abstract Bhim Singh",picspath.joinpath('lsa_abstractSingh.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,dataSingh.Abstract),"LDA Abstract Bhim Singh",picspath.joinpath('lda_abstractSingh.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [13]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,dataLi.Title),"NMF Title Liang Li",picspath.joinpath('nmf_titleLi.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,dataLi.Title),"LSA Title Liang Li",picspath.joinpath('lsa_titleLi.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,dataLi.Title),"LDA Title Liang Li",picspath.joinpath('lda_titleLi.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,dataLi.Abstract),"NMF Abstract Liang Li",picspath.joinpath('nmf_abstractLi.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,dataLi.Abstract),"LSA Abstract Liang Li",picspath.joinpath('lsa_abstractLi.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,dataLi.Abstract),"LDA Abstract Liang Li",picspath.joinpath('lda_abstractLi.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [29]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,dataMeyer.Title),"NMF Title J-P. Meyer",picspath.joinpath('nmf_titleMeyer.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,dataMeyer.Title),"LSA Title J-P. Meyer",picspath.joinpath('lsa_titleMeyer.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,dataMeyer.Title),"LDA Title J-P. Meyer",picspath.joinpath('lda_titleMeyer.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,dataMeyer.Abstract),"NMF Abstract J-P. Meyer",picspath.joinpath('nmf_abstractMeyer.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,dataMeyer.Abstract),"LSA Abstract J-P. Meyer",picspath.joinpath('lsa_abstractMeyer.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,dataMeyer.Abstract),"LDA Abstract J-P. Meyer",picspath.joinpath('lda_abstractMeyer.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


# vis w/ filtered data: country

In [30]:
data[data.groupby('Source Country')['Source Country'].transform('count')>1000]['Source Country'].value_counts()

Source Country
United Kingdom    44550
United States     35407
Germany           18065
Switzerland       12515
Netherlands       11835
China              1795
Egypt              1337
India              1097
Name: count, dtype: int64

In [11]:
dataUK=data[data['Source Country']=='United Kingdom']
dataGer=data[data['Source Country']=='Germany']
dataIn=data[data['Source Country']=='India']

In [32]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,dataUK.Title),"NMF Title United Kingdom",picspath.joinpath('nmf_titleUK.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,dataUK.Title),"LSA Title United Kingdom",picspath.joinpath('lsa_titleUK.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,dataUK.Title),"LDA Title United Kingdom",picspath.joinpath('lda_titleUK.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,dataUK.Abstract),"NMF Abstract United Kingdom",picspath.joinpath('nmf_abstractUK.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,dataUK.Abstract),"LSA Abstract United Kingdom",picspath.joinpath('lsa_abstractUK.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,dataUK.Abstract),"LDA Abstract United Kingdom",picspath.joinpath('lda_abstractUK.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [33]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,dataGer.Title),"NMF Title Germany",picspath.joinpath('nmf_titleGer.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,dataGer.Title),"LSA Title Germany",picspath.joinpath('lsa_titleGer.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,dataGer.Title),"LDA Title Germany",picspath.joinpath('lda_titleGer.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,dataGer.Abstract),"NMF Abstract Germany",picspath.joinpath('nmf_abstractGer.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,dataGer.Abstract),"LSA Abstract Germany",picspath.joinpath('lsa_abstractGer.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,dataGer.Abstract),"LDA Abstract Germany",picspath.joinpath('lda_abstractGer.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [12]:
vis_w_matrix(get_topic_destribution(nmf_titles_model,dataIn.Title),"NMF Title India",picspath.joinpath('nmf_titleIn.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_titles_model,dataIn.Title),"LSA Title India",picspath.joinpath('lsa_titleIn.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_titles_model,dataIn.Title),"LDA Title India",picspath.joinpath('lda_titleIn.png').as_posix())

vis_w_matrix(get_topic_destribution(nmf_abstracts_model,dataIn.Abstract),"NMF Abstract India",picspath.joinpath('nmf_abstractIn.png').as_posix())
vis_w_matrix(get_topic_destribution(lsa_abstracts_model,dataIn.Abstract),"LSA Abstract India",picspath.joinpath('lsa_abstractIn.png').as_posix())
vis_w_matrix(get_topic_destribution(lda_abstracts_model,dataIn.Abstract),"LDA Abstract India",picspath.joinpath('lda_abstractIn.png').as_posix())

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


# get topic list as file

In [34]:
modelvispath=pathlib.Path.cwd().joinpath('models/modelvis')
def get_topic_table(model,file,m):
    num_words=5
    H=model.named_steps[m].components_
    feature_names = model.named_steps['tfidf'].get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(H):
        topics[f"Topic {topic_idx}"] = [feature_names[i] for i in topic.argsort()[:-num_words-1:-1]]
    topics_df = pd.DataFrame(topics)
    topics_df.to_csv(file+'.csv', index=False)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=topics_df.values, colLabels=topics_df.columns, cellLoc='center', loc='center')
    plt.savefig(file+'.pdf')
    plt.close()


In [35]:
get_topic_table(nmf_titles_model,modelvispath.joinpath('nmf_titles').as_posix(),'nmf')
get_topic_table(nmf_abstracts_model,modelvispath.joinpath('nmf_abstracts').as_posix(),'nmf')

get_topic_table(lsa_titles_model,modelvispath.joinpath('lsa_titles').as_posix(),'lsa')
get_topic_table(lsa_abstracts_model,modelvispath.joinpath('lsa_abstracts').as_posix(),'lsa')

get_topic_table(lda_titles_model,modelvispath.joinpath('lda_titles').as_posix(),'lda')
get_topic_table(lda_abstracts_model,modelvispath.joinpath('lda_abstracts').as_posix(),'lda')

In [36]:
lda_titles_model.steps

[('tfidf',
  TfidfVectorizer(max_df=0.6731187135180453, min_df=99, stop_words='english',
                  tokenizer=<function tokenize at 0x0000015107D20360>)),
 ('lda', LatentDirichletAllocation(n_components=50, random_state=313))]